# Install and Import Libraries

In [217]:
# !pip install pdfplumber -q
# !pip install sentence-transformers
# !pip install scikit-learn


In [218]:
# !conda install -c conda-forge scikit-surprise -y
!pip install scikit-suprise

Defaulting to user installation because normal site-packages is not writeable

ERROR: Could not find a version that satisfies the requirement scikit-suprise (from versions: none)
ERROR: No matching distribution found for scikit-suprise


In [219]:
import pdfplumber
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords 
from string import punctuation
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
from heapq import nlargest
from collections import defaultdict
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.neighbors import NearestNeighbors
from sentence_transformers import SentenceTransformer
from nltk.collocations import *
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vedan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vedan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vedan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\vedan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

# Below Code is to convert Resume Pdf to Text

In [220]:
# input_file_path = "/Users/khedved/VEDANG/Documents/yash.pdf"
# output_file_path = "/Users/khedved/VEDANG/Documents/Vedang_Khedekar_2025.txt"

# with pdfplumber.open(input_file_path) as pdf:
#     text = " ".join(page.extract_text(x_tolerance=2).replace("\n", " ") for page in pdf.pages)

# with open(output_file_path, "w", encoding="utf-8") as file:
#     file.write(text)
# text
current_path='E:/ubuntu/VEDANG_STUDIES/SEM8/BTECH_PROJECT/HELPER/final2/'
# current_path='/Users/khedved/VEDANG/Projects/JobGenix-A-Job-and-Talent-Recommendation-Platform/'

# Loading locations and Stop-words

In [221]:
# Locations
f1=open(current_path+'in_districts.txt','r', errors = 'ignore')
text1=f1.read()
locations = word_tokenize(text1.replace("\n", " "))

# Generate stopwords.txt file
stop_words=set(stopwords.words('english'))
with open(current_path+'stopwords.txt',"w") as f:
    for word in stop_words:
            f.write(word+"\n")

# Additional stopwords
f2=open(current_path+'stopwords.txt','r', errors = 'ignore')
text2=f2.read()
stopwords_additional = word_tokenize(text2.replace("\n", " "))
print("locations and stopwords loaded")
# print("locations: ",locations)
# print("stopwords_additional: ",stopwords_additional)

locations and stopwords loaded


# Loading function and apply them to clean resume data

In [222]:
# Below function is used to apply natural language processing on resumes
def nlp(text):
    """Tokenizes, removes stopwords, and lemmatizes the given text."""
    if pd.isna(text):  # Handle NaN values
        return ""
    
    text = text.lower().replace("\n", " ")  # Normalize text
    words = word_tokenize(text)

    # Define stopwords
    stop_words = set(stopwords.words('english') + list(punctuation) + list("●") + 
                     list('–') + list('’') + locations + stopwords_additional)
    
    # Remove stopwords
    words = [word for word in words if word not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    processed_text = [lemmatizer.lemmatize(word) for word in words]
    
    return processed_text  # Returning list of words for further processing


In [223]:
# Below function is used to get key words from resumes
def get_key_word(tokens):
    """Finds repeated bigrams (two-word phrases) in tokenized text."""
    finder = BigramCollocationFinder.from_words(tokens)
    key_phrases = []

    for bigram, count in finder.ngram_fd.items():
        if count > 1:  # Only keep phrases appearing more than once
            key_phrases.append(" ".join(bigram))
    
    return key_phrases


In [224]:
# Load cleaned resume dataset
file_path = current_path+"cleaned_file.csv"
df = pd.read_csv(file_path)

# Display first few rows
df.head()


,user_id,resume_id,resume
0,1,RES_001,SUMMARY Certified QA tester with more than 8 y...
1,506,RES_506,SUMMARY Over 7 yearsâ experience in IT indus...
2,518,RES_518,SUMMARY +7 years of extensive experience in So...
3,508,RES_508,Vedang Khedekar khedekarvp21.comp@coeptech.ac....
4,2,RES_002,Sandesh Sahane 7340414303 | sandeshssahane23@g...


In [225]:
# Apply NLP function to the "resume" column
df["Processed_Resume"] = df["resume"].astype(str).apply(nlp)

# Save processed resume data
processed_resume_csv =current_path+ "processed_resume_data.csv"
df.to_csv(processed_resume_csv, index=False)

print(f"Processed resumes saved to: {processed_resume_csv}")

# Display first few rows of processed data
df.head()


Processed resumes saved to: E:/ubuntu/VEDANG_STUDIES/SEM8/BTECH_PROJECT/HELPER/final2/processed_resume_data.csv


,user_id,resume_id,resume,Processed_Resume
0,1,RES_001,SUMMARY Certified QA tester with more than 8 y...,"[summary, certified, qa, tester, 8, year, prov..."
1,506,RES_506,SUMMARY Over 7 yearsâ experience in IT indus...,"[summary, 7, yearsâ, experience, industry, 5..."
2,518,RES_518,SUMMARY +7 years of extensive experience in So...,"[summary, +7, year, extensive, experience, sof..."
3,508,RES_508,Vedang Khedekar khedekarvp21.comp@coeptech.ac....,"[vedang, khedekar, khedekarvp21.comp, coeptech..."
4,2,RES_002,Sandesh Sahane 7340414303 | sandeshssahane23@g...,"[sandesh, sahane, 7340414303, sandeshssahane23..."


In [226]:
# Apply keyword extraction
df["Key_Resume"] = df["Processed_Resume"].apply(get_key_word)

# Save key resume data
key_resume_csv = current_path+"key_resume_data.csv"
df.to_csv(key_resume_csv, index=False)

print(f"Key resume data saved to: {key_resume_csv}")

# Display first few rows with key phrases
df.head()

Key resume data saved to: E:/ubuntu/VEDANG_STUDIES/SEM8/BTECH_PROJECT/HELPER/final2/key_resume_data.csv


,user_id,resume_id,resume,Processed_Resume,Key_Resume
0,1,RES_001,SUMMARY Certified QA tester with more than 8 y...,"[summary, certified, qa, tester, 8, year, prov...","[qa tester, quality assurance, maintain test, ..."
1,506,RES_506,SUMMARY Over 7 yearsâ experience in IT indus...,"[summary, 7, yearsâ, experience, industry, 5...","[sql server, operating system, database backup..."
2,518,RES_518,SUMMARY +7 years of extensive experience in So...,"[summary, +7, year, extensive, experience, sof...","[testing using, soap ui, manual testing, quali..."
3,508,RES_508,Vedang Khedekar khedekarvp21.comp@coeptech.ac....,"[vedang, khedekar, khedekarvp21.comp, coeptech...","[vedang khedekar, khedekar khedekarvp21.comp, ..."
4,2,RES_002,Sandesh Sahane 7340414303 | sandeshssahane23@g...,"[sandesh, sahane, 7340414303, sandeshssahane23...","[web android, android portfolio, pattern techn..."


# Import Job Data Set

In [227]:
#Import scraped data from souce 1
# df = pd.read_csv("E:/ubuntu/VEDANG_STUDIES/SEM8/BTECH_PROJECT/HELPER/india_jobs_dummy.csv")
df = pd.read_csv(current_path+"india_jobs_dummy.csv")

df.head(2)

,JobID,title,company,experience,salary,location,job description,skills,post_date,webpage,company overview,industry,job highlights,All
0,1000,Cybersecurity Analyst,IBM India,5-7 years,INR 600000-1200000 per annum,Chennai,Provide security awareness training. Monitor s...,Secure Coding Risk Assessment Incident Respons...,2025-01-12,http://www.bala.com/,"IBM India offers AI, cloud computing, and ente...",E-commerce,Participate in hackathons and innovation chall...,Cybersecurity Analyst IBM India Chennai Partic...
1,1001,Machine Learning Engineer,Tech Mahindra,7-10 years,INR 2500000-3500000 per annum,NaN,Improve model accuracy with hyperparameter tun...,"Big Data Frameworks (Spark, Hadoop) Supervised...",2025-02-18,http://www.loyal.biz/,Tech Mahindra offers IT services and business ...,Finance,Participate in hackathons and innovation chall...,Machine Learning Engineer Tech Mahindra Parti...


In [228]:
df.shape

(1200, 14)

In [229]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   JobID             1200 non-null   int64 
 1   title             1200 non-null   object
 2   company           1200 non-null   object
 3   experience        1200 non-null   object
 4   salary            1200 non-null   object
 5   location          951 non-null    object
 6   job description   1200 non-null   object
 7   skills            1200 non-null   object
 8   post_date         1200 non-null   object
 9   webpage           1200 non-null   object
 10  company overview  1200 non-null   object
 11  industry          1200 non-null   object
 12  job highlights    1200 non-null   object
 13  All               1200 non-null   object
dtypes: int64(1), object(13)
memory usage: 131.4+ KB


In [230]:
# Load the dataset and update the All field
file_path = current_path+"india_jobs_dummy.csv"
df = pd.read_csv(file_path)

# Fill NaN values with empty strings to avoid issues while concatenating
df.fillna("", inplace=True)

# Combine required columns into a single "All" column
df["All"] = df["title"] + " " + df["company"] + " " + df["location"] + " " + \
            df["job highlights"] + " " + df["job description"] + " " + df["skills"] +" "+ \
            df["company overview"] + " " + df["industry"] + " " + df["experience"]

# Save the updated DataFrame back to CSV
df.to_csv(file_path, index=False)

print("CSV file updated successfully!")
df.info()



CSV file updated successfully!
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   JobID             1200 non-null   int64 
 1   title             1200 non-null   object
 2   company           1200 non-null   object
 3   experience        1200 non-null   object
 4   salary            1200 non-null   object
 5   location          1200 non-null   object
 6   job description   1200 non-null   object
 7   skills            1200 non-null   object
 8   post_date         1200 non-null   object
 9   webpage           1200 non-null   object
 10  company overview  1200 non-null   object
 11  industry          1200 non-null   object
 12  job highlights    1200 non-null   object
 13  All               1200 non-null   object
dtypes: int64(1), object(13)
memory usage: 131.4+ KB


# Testing with scrapped data

In [231]:
# Load the dataset and update the All field
file_path = current_path+"naukri_final_scraping_2.csv"
df2 = pd.read_csv(file_path)

# Fill NaN values with empty strings to avoid issues while concatenating
df2.fillna("", inplace=True)

# Combine required columns into a single "All" column

df2["All"] = df2["title"] + " " + df2["company"] + " " + df2["location"] + " " + \
            df2["job highlights"] + " " + df2["job description"] + " " + df2["skills"] +" " + \
            df2["company overview"] + " " + df2["industry"] + " " + df2["experience"]

# df2=df2.drop(columns=['data source'])
# Save the updated DataFrame back to CSV
df2.to_csv(file_path, index=False)

print("CSV file updated successfully!")
df2.info()


CSV file updated successfully!
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1249 entries, 0 to 1248
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   JobID             1249 non-null   int64 
 1   title             1249 non-null   object
 2   company           1249 non-null   object
 3   experience        1249 non-null   object
 4   salary            1249 non-null   object
 5   location          1249 non-null   object
 6   job description   1249 non-null   object
 7   skills            1249 non-null   object
 8   post_date         1249 non-null   object
 9   webpage           1249 non-null   object
 10  company overview  1249 non-null   object
 11  industry          1249 non-null   object
 12  job highlights    1249 non-null   object
 13  All               1249 non-null   object
dtypes: int64(1), object(13)
memory usage: 136.7+ KB


# Merge generated and scrapped Data

In [232]:
df_combined=pd.concat([df,df2],ignore_index=True)
df_combined['index']=df_combined.index
df_combined.to_csv(current_path+"combined.csv",index=False)
df_combined

,JobID,title,company,experience,salary,location,job description,skills,post_date,webpage,company overview,industry,job highlights,All,index
0,1000,Cybersecurity Analyst,IBM India,5-7 years,INR 600000-1200000 per annum,Chennai,Provide security awareness training. Monitor s...,Secure Coding Risk Assessment Incident Respons...,2025-01-12,http://www.bala.com/,"IBM India offers AI, cloud computing, and ente...",E-commerce,Participate in hackathons and innovation chall...,Cybersecurity Analyst IBM India Chennai Partic...,0
1,1001,Machine Learning Engineer,Tech Mahindra,7-10 years,INR 2500000-3500000 per annum,,Improve model accuracy with hyperparameter tun...,"Big Data Frameworks (Spark, Hadoop) Supervised...",2025-02-18,http://www.loyal.biz/,Tech Mahindra offers IT services and business ...,Finance,Participate in hackathons and innovation chall...,Machine Learning Engineer Tech Mahindra Parti...,1
2,1002,Backend Developer,Google India,7-10 years,INR 1200000-1800000 per annum,,Write clean and maintainable backend code. Ens...,Serverless Architecture Containerization (Dock...,2025-01-01,http://www.misra.com/,"Google India focuses on search, cloud computin...",Healthcare,Work closely with product managers to enhance ...,Backend Developer Google India Work closely w...,2
3,1003,Product Manager,Accenture,5-7 years,INR 2500000-3500000 per annum,Pune,Work with stakeholders for strategic planning....,Product Strategy User Experience (UX) Competit...,2025-01-30,https://www.mallick.com/,Accenture is a global professional services co...,Automobile,Competitive salary package with performance-ba...,Product Manager Accenture Pune Competitive sal...,3
4,1004,Software Engineer,SAP Labs India,1-3 years,INR 1200000-1800000 per annum,,Integrate third-party services and APIs. Devel...,Object-Oriented Programming Problem Solving Te...,2025-01-23,https://www.dasgupta-mane.net/,SAP Labs India focuses on enterprise software ...,Finance,Work on cutting-edge technologies in a fast-pa...,Software Engineer SAP Labs India Work on cutt...,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2444,90125504932,Software Developer 2,Oracle,3 - 5 years,Not Disclosed,Bengaluru,Oracle Analytics is a complete platform with r...,"Unix, Automation, Coding, Debugging, Perl, Win...",30+ Days Ago,https://www.naukri.com/job-listings-software-d...,Not Available,Not Available,Not Available,Software Developer 2 Oracle Bengaluru Not Avai...,2444
2445,90125504948,Software Developer 3 - Web Technologies,Oracle,3 - 5 years,Not Disclosed,Bengaluru,Department DescriptionThe worlds most complete...,"Product management, Analytical, Debugging, Jav...",30+ Days Ago,https://www.naukri.com/job-listings-software-d...,Not Available,Not Available,Not Available,Software Developer 3 - Web Technologies Oracle...,2445
2446,120225502409,Embedded Software Developer,Roots Group of Companies,3 - 4 years,Not Disclosed,Coimbatore,1.Proficient in embedded software development ...,"Project development, C++, C, Simulation, ISO, ...",18 Days Ago,https://www.naukri.com/job-listings-embedded-s...,Not Available,Not Available,Not Available,Embedded Software Developer Roots Group of Com...,2446
2447,70225012187,Associate Software Developer,TEKsystems,3 - 6 years,Not Disclosed,Hyderabad,"Mandatory skillsReactJs, HTML, CSS, JavaScript...","ReactJs, NextJS, TypeScript, CSS, JavaScript, ...",23 Days Ago,https://www.naukri.com/job-listings-associate-...,A US-based $4.2 billion renowned brand in IT t...,Not Available,Not Available,Associate Software Developer TEKsystems Hydera...,2447


# Get Recommendation function to get to N jobs

In [233]:
def get_recommendation(top, df, scores):
    recommendation = pd.DataFrame(columns=['JobID', 'title', 'industry', 'job description', 'score'])
    for count, i in enumerate(top):
        recommendation.at[count, 'JobID'] = df.index[i]
        recommendation.at[count, 'title'] = df['title'][i]
        recommendation.at[count, 'industry'] = df['industry'][i]
        recommendation.at[count, 'job description'] = df['job description'][i]
        recommendation.at[count, 'score'] = scores[count]
    return recommendation


# Algorithms to calculate cosine scores

In [234]:
# TFIDF function to calculate cosine simmilarity
def TFIDF(scraped_data, cv):
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_jobid = tfidf_vectorizer.fit_transform(scraped_data)
    user_tfidf = tfidf_vectorizer.transform(cv)
    cos_similarity_tfidf = map(lambda x: cosine_similarity(user_tfidf, x), tfidf_jobid)
    return list(cos_similarity_tfidf)

In [235]:
# Count Vectorization function to calculate cosine simmilarity
def count_vectorize(scraped_data, cv):
    count_vectorizer = CountVectorizer()
    count_jobid = count_vectorizer.fit_transform(scraped_data)
    user_count = count_vectorizer.transform(cv)
    cos_similarity_countv = map(lambda x: cosine_similarity(user_count, x), count_jobid)
    return list(cos_similarity_countv)

In [236]:
# KNN function to calculate top n recommendations
def KNN(scraped_data, cv, n_neighbors=100):
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    job_tfidf_matrix = tfidf_vectorizer.fit_transform(scraped_data)  # Fit on job data
    cv_tfidf_matrix = tfidf_vectorizer.transform(cv)  # Transform candidate CV

    # Fit KNN model
    knn_model = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine')
    knn_model.fit(job_tfidf_matrix)

    # Find nearest neighbors
    NNs = knn_model.kneighbors(cv_tfidf_matrix, return_distance=True)
    top_indices = NNs[1][0]  # Top job indices
    similarity_scores = 1 - NNs[0][0]  # Convert distance to similarity score (cosine)

    # Get job recommendations
    knn_recommendations = get_recommendation(top_indices, df_combined, similarity_scores)
    return knn_recommendations

In [237]:
# Load data set in resume data frame
resume_df = pd.read_csv(current_path+"key_resume_data.csv")

# TFIDF

In [238]:
# This cell calls TFIDF for each resume and job data set
final_recommendations = pd.DataFrame()
all_recommendations_TFIDF = []

for index, row in resume_df.iterrows():
    user_id = row['user_id']
    resume_text = row['Processed_Resume']
    similarity_scores = TFIDF(df_combined['All'], [resume_text])
    top = sorted(range(len(similarity_scores)), key=lambda i: similarity_scores[i], reverse=True)[:100]
    list_scores = [similarity_scores[i][0][0] for i in top]
    recommendation = get_recommendation(top, df_combined, list_scores)
    recommendation["user_id"] = user_id
    all_recommendations_TFIDF.append(recommendation)
    # break  # Process only the first resume for debugging
print("TFIDF Calculated")

TFIDF Calculated


In [239]:
# Store and display Recommendations using TFIDF

# final_recommendations = pd.concat(all_recommendations_TFIDF, ignore_index=True)
# final_recommendations.to_csv("user_recommendations_tfidf.csv", index=False)
print(all_recommendations_TFIDF)
print("Recommendations saved successfully for TFIDF.")


[   JobID                                          title       industry  \
0   1688            Software Developer Engineer In Test  Not Available   
1   1539                           Software Developer 3  Not Available   
2   1535                           Software Developer 4  Not Available   
3   1748            Software Developer Engineer in Test  Not Available   
4   1516                           Software Developer 3  Not Available   
..   ...                                            ...            ...   
95  2424                           Software Developer 3  Not Available   
96  1964                             Software Developer  Not Available   
97  2302        IBM Public Cloud CTO Software Developer  Not Available   
98  1743  Software Developer Engineer For Testing(SDET)  Not Available   
99  1243                             Software developer  Not Available   

                                      job description     score  user_id  
0   As Full Stack Software Test Aut

# CV

In [240]:
# This cell calls Count Vectorization for each resume and job data set
all_recommendations_CV = []

for index, row in resume_df.iterrows():
    
    user_id = row['user_id']
    resume_text = row['Processed_Resume']
    similarity_scores = count_vectorize(df_combined['All'], [resume_text])
    top = sorted(range(len(similarity_scores)), key=lambda i: similarity_scores[i], reverse=True)[:100]
    list_scores = [similarity_scores[i][0][0] for i in top]
    recommendation = get_recommendation(top, df_combined, list_scores)
    recommendation["user_id"] = user_id
    all_recommendations_CV.append(recommendation)
    # break  # Process only the first resume for debugging
print("Count Vectorization Calculated")

Count Vectorization Calculated


In [241]:
# Store and display Recommendations using Count Vectorization

# final_recommendations = pd.concat(all_recommendations_CV, ignore_index=True)
# final_recommendations.to_csv("user_recommendations_tfidf.csv", index=False)
print(all_recommendations_CV)
print("Recommendations saved successfully for Count Vectorization.")


[   JobID                                    title       industry  \
0   1688      Software Developer Engineer In Test  Not Available   
1   1520  Software Developer Engineer in Test - I  Not Available   
2   1535                     Software Developer 4  Not Available   
3   1539                     Software Developer 3  Not Available   
4   1240           Software Developer Engineer II  Not Available   
..   ...                                      ...            ...   
95  2381             Software Developer (ASP.Net)  Not Available   
96  1523   Senior Software Developer - Automation  Not Available   
97   289                           Data Scientist     Automobile   
98   730                        Software Engineer     Healthcare   
99  1567                       Software Developer  Not Available   

                                      job description     score  user_id  
0   As Full Stack Software Test Automation Enginee...   0.32463        1  
1   ResponsibilitiesManage the e

# KNN

In [242]:
# This cell calls KNN for each resume and job data set
all_recommendations_KNN = []

for index, row in resume_df.iterrows():
    user_id = row['user_id']
    resume_text = row['Processed_Resume']
    recommendation = KNN(df_combined['All'], [resume_text])
    recommendation["user_id"] = user_id
    all_recommendations_KNN.append(recommendation)
    # break  # Process only the first resume for debugging
print("KNN Calculated")

KNN Calculated


In [243]:
# Store and display Recommendations using KNN

# final_recommendations = pd.concat(all_recommendations_KNN, ignore_index=True)
# final_recommendations.to_csv("user_recommendations_tfidf.csv", index=False)
print(all_recommendations_KNN)
print("Recommendations saved successfully for KNN.")


[   JobID                                          title       industry  \
0   1688            Software Developer Engineer In Test  Not Available   
1   1539                           Software Developer 3  Not Available   
2   1535                           Software Developer 4  Not Available   
3   1748            Software Developer Engineer in Test  Not Available   
4   1516                           Software Developer 3  Not Available   
..   ...                                            ...            ...   
95  2424                           Software Developer 3  Not Available   
96  1964                             Software Developer  Not Available   
97  2302        IBM Public Cloud CTO Software Developer  Not Available   
98  1743  Software Developer Engineer For Testing(SDET)  Not Available   
99  1243                             Software developer  Not Available   

                                      job description     score  user_id  
0   As Full Stack Software Test Aut

# Display TFIDF, CV and KNN result together

In [244]:

# Convert lists of recommendations into DataFrames
df_TFIDF = pd.concat(all_recommendations_TFIDF, ignore_index=True)
df_CV = pd.concat(all_recommendations_CV, ignore_index=True)
df_KNN = pd.concat(all_recommendations_KNN, ignore_index=True)
# df_BERT = pd.concat(all_recommendations_BERT, ignore_index=True)

# Select only relevant columns before merging
df_TFIDF = df_TFIDF[['user_id', 'JobID', 'score']].rename(columns={'score': 'score_TFIDF'})
df_CV = df_CV[['user_id', 'JobID', 'score']].rename(columns={'score': 'score_CV'})
df_KNN = df_KNN[['user_id', 'JobID', 'score']].rename(columns={'score': 'score_KNN'})
# df_BERT = df_BERT[['user_id', 'JobID', 'score']].rename(columns={'score': 'score_BERT'})

# Merge based on common 'user_id' and 'job_id'
merged_df = df_TFIDF.merge(df_CV, on=["user_id", "JobID"], how="inner")
merged_df = merged_df.merge(df_KNN, on=["user_id", "JobID"], how="inner")
# merged_df = merged_df.merge(df_BERT, on=["user_id", "JobID"], how="inner")
# merged_df = df_CV.merge(df_BERT, on=["user_id", "JobID"], how="inner")

# Add BERT scores separately using LEFT JOIN
# merged_df = merged_df.merge(df_BERT, on=["user_id", "JobID"], how="left")

# Sort results for each user based on the highest TF-IDF score (or any preferred metric)
# merged_df = merged_df.sort_values(by=["user_id", "score_TFIDF"], ascending=[True, False])

# Display the final merged recommendations
print(merged_df.head())

# Save to CSV if needed
merged_df.to_csv("final_recommendations.csv", index=False)

   user_id JobID score_TFIDF  score_CV score_KNN
0        1  1688    0.292058   0.32463  0.292058
1        1  1539    0.239966  0.243797  0.239966
2        1  1535    0.239777  0.243887  0.239777
3        1  1748     0.23718  0.229168   0.23718
4        1  1516    0.219485  0.202462  0.219485


# BERT


In [245]:
# Load Pretrained Sentence Transformer Model
bert_model = SentenceTransformer('all-MiniLM-L6-v2')
def BERT(scraped_data, cv, top_n=200):
    # Encode Job Descriptions
    job_embeddings = bert_model.encode(scraped_data, convert_to_tensor=True)

    # Encode Candidate's CV
    cv_embedding = bert_model.encode(cv, convert_to_tensor=True)

    # Compute Cosine Similarity
    similarities = np.dot(job_embeddings, cv_embedding.T).flatten()

    # Get Top Matches
    top_indices = similarities.argsort()[-top_n:][::-1]  # Top 200 jobs

    # Get Recommendations
    bert_recommendations = get_recommendation_bert(top_indices, df_combined, similarities[top_indices])

    return bert_recommendations

def get_recommendation_bert(top, df, scores):
    recommendation = pd.DataFrame(columns=['JobID', 'title', 'industry', 'job description', 'score'])
    for count, i in enumerate(top):
        recommendation.at[count, 'JobID'] = df.iloc[i]['JobID']  # Fix: Use iloc to get the correct JobID
        recommendation.at[count, 'title'] = df.iloc[i]['title']
        recommendation.at[count, 'industry'] = df.iloc[i]['industry']
        recommendation.at[count, 'job description'] = df.iloc[i]['job description']
        recommendation.at[count, 'score'] = scores[count]
    return recommendation


In [246]:
all_recommendations_BERT = []
for index, row in merged_df.iterrows():
    user_id = row['user_id']
    job_id = row['JobID']

    # Get Job Description for the given JobID
    job_text = df_combined[df_combined.index == job_id]['All'].values

    # Get Resume for the User
    resume_text = resume_df[resume_df['user_id'] == user_id]['Processed_Resume'].values

    if len(job_text) > 0 and len(resume_text) > 0:
        recommendation = BERT(job_text, [resume_text[0]], top_n=1)  # Apply BERT for only this job-user pair
        recommendation["user_id"] = user_id
        recommendation["JobID"] = job_id
        all_recommendations_BERT.append(recommendation)

print("BERT Applied to Merged Jobs and Users")


BERT Applied to Merged Jobs and Users


In [247]:
df_BERT = pd.concat(all_recommendations_BERT, ignore_index=True)
df_BERT = df_BERT[['user_id', 'JobID', 'score']].rename(columns={'score': 'score_BERT'})
# print(df_BERT.head())
# Merge BERT scores with the existing merged DataFrame
final_df = merged_df.merge(df_BERT, on=["user_id", "JobID"], how="left")

# Save Final Results
final_df.to_csv("final_recommendations_with_BERT.csv", index=False)

# Display Final DataFrame
print(final_df.head())


   user_id JobID score_TFIDF  score_CV score_KNN score_BERT
0        1  1688    0.292058   0.32463  0.292058   0.511246
1        1  1539    0.239966  0.243797  0.239966     0.2902
2        1  1535    0.239777  0.243887  0.239777   0.287746
3        1  1748     0.23718  0.229168   0.23718   0.453017
4        1  1516    0.219485  0.202462  0.219485   0.342026


# Scale and assign weights

In [300]:
# Scale it
final_df=pd.read_csv('final_recommendations_with_BERT.csv')
from sklearn.preprocessing import MinMaxScaler
# slr = MinMaxScaler()
# final[["KNN", "TF-IDF", 'CV']] = slr.fit_transform(final[["KNN", "TF-IDF", 'CV']])
# print(final['KNN'])
# Multiply by weights
a=final_df['score_KNN']*0.2
b=final_df['score_TFIDF']*0.1
c=final_df['score_CV']*0.1
d=final_df['score_BERT']
# print(a)
# print(b)
# print(c)

# final['KNN'] = a
# final['TF-IDF'] = b
# final['CV'] = c
final_df['Final'] = d

final_df = final_df.sort_values(by=["user_id", "Final"], ascending=[True, False])

final_df.to_csv("final.csv", index=False)
print(final_df)
# final.sort_values(by="Final", ascending=False)

     user_id  JobID  score_TFIDF  score_CV  score_KNN  score_BERT     Final
12         1   2350     0.185272  0.220928   0.185272    0.544655  0.544655
0          1   1688     0.292058  0.324630   0.292058    0.511246  0.511246
19         1   2444     0.177338  0.179962   0.177338    0.486068  0.486068
52         1   1743     0.124877  0.175399   0.124877    0.484760  0.484760
33         1   2321     0.152668  0.190309   0.152668    0.468483  0.468483
..       ...    ...          ...       ...        ...         ...       ...
166      518    280     0.147100  0.222948   0.147100    0.321553  0.321553
180      518    799     0.134055  0.197446   0.134055    0.315896  0.315896
169      518    239     0.145643  0.187183   0.145643    0.289819  0.289819
146      518   1378     0.185381  0.171158   0.185381    0.275674  0.275674
182      518   1732     0.132097  0.194162   0.132097    0.267861  0.267861

[382 rows x 7 columns]


# Return Top 10 jobs for each user

In [305]:
# Group by user_id and get the top 10 recommendations for each user
top_10 = final_df.groupby("user_id").head(10)

# Sort the final result by user_id and Final score
top_10 = top_10.sort_values(by=["user_id", "Final"], ascending=[True, False])

# Display the result
print(top_10)

# Save to CSV
top_10.to_csv("top_10_recommendations.csv", index=False)


     user_id  JobID  score_TFIDF  score_CV  score_KNN  score_BERT     Final  \
1          1   2350     0.405462  0.400354   0.405462    0.544655  0.544655   
0          1   1688     0.715702  0.701930   0.715702    0.511246  0.511246   
3          1   2444     0.382411  0.281218   0.382411    0.486068  0.486068   
6          1   1743     0.229998  0.267950   0.229998    0.484760  0.484760   
5          1   2321     0.310738  0.311311   0.310738    0.468483  0.468483   
..       ...    ...          ...       ...        ...         ...       ...   
333      518   1743     0.330458  0.385199   0.330458    0.487797  0.487797   
338      518   1971     0.253585  0.307218   0.253585    0.471326  0.471326   
335      518   2281     0.370318  0.329338   0.370318    0.470391  0.470391   
330      518   2350     0.488469  0.415928   0.488469    0.464835  0.464835   
336      518   1855     0.439106  0.248646   0.439106    0.462850  0.462850   

     actual_application  
1                     1  

In [302]:
# Ensure JobID in top_10 is of the same type as df_combined index
top_10["JobID"] = top_10["JobID"].astype(str)

# Convert index of df_combined to string and reset index
df_combined.index = df_combined.index.astype(str)

# Merge top_10 with df_combined using index as JobID
top_10_with_details = top_10.merge(df_combined, left_on="JobID", right_index=True, how="left")

# Display the result
print(top_10_with_details)

# Save to CSV if needed
top_10_with_details.to_csv("top_10_recommendations_with_details.csv", index=False)


    JobID  user_id JobID_x  score_TFIDF  score_CV  score_KNN  score_BERT  \
12   2350        1    2350     0.185272  0.220928   0.185272    0.544655   
0    1688        1    1688     0.292058  0.324630   0.292058    0.511246   
19   2444        1    2444     0.177338  0.179962   0.177338    0.486068   
52   1743        1    1743     0.124877  0.175399   0.124877    0.484760   
33   2321        1    2321     0.152668  0.190309   0.152668    0.468483   
..    ...      ...     ...          ...       ...        ...         ...   
159  1743      518    1743     0.159456  0.215717   0.159456    0.487797   
181  1971      518    1971     0.132996  0.188902   0.132996    0.471326   
150  2281      518    2281     0.173176  0.196509   0.173176    0.470391   
134  2350      518    2350     0.213844  0.226284   0.213844    0.464835   
138  1855      518    1855     0.196853  0.168761   0.196853    0.462850   

        Final       JobID_y  \
12   0.544655  190225501627   
0    0.511246  2702255007

# Algorithms for recommending Resumes to Recruiters

In [251]:
def get_recommendation_rev(top, df, scores):
    # Create a list of dictionaries for better performance
    recommendation = pd.DataFrame(columns=['JobID','user_id', 'resume_id','score'])
    
    for count, i in enumerate(top):
        recommendation.at[count, 'user_id'] = df.iloc[i]['user_id']
        recommendation.at[count, 'resume_id'] = df.iloc[i]['resume_id']  # Optional, if available
        recommendation.at[count, 'score'] = scores[count]
    
    return recommendation

In [252]:
def KNN_rev(resume_data, job_desc, n_neighbors=8):
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')

    # Fit on all resumes (instead of jobs)
    resume_tfidf_matrix = tfidf_vectorizer.fit_transform(resume_data)
    
    # Transform the given job description using the same vectorizer
    job_tfidf_matrix = tfidf_vectorizer.transform(job_desc)

    # Fit KNN model on resumes
    knn_model = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine')
    knn_model.fit(resume_tfidf_matrix)

    # Find nearest resumes
    NNs = knn_model.kneighbors(job_tfidf_matrix, return_distance=True)
    top_indices = NNs[1][0]  # Top resume indices
    similarity_scores = 1 - NNs[0][0]  # Convert distance to similarity score (cosine similarity)

    # Get resume recommendations
    knn_recommendations = get_recommendation_rev(top_indices, resume_df, similarity_scores)
    return knn_recommendations


# TFIDF for Recruiter

In [253]:
# This cell calls TFIDF for each job to recommend resumes
final_recommendations_rev = pd.DataFrame()
all_recommendations_TFIDF_rev = []

for index, row in df_combined.iterrows():  
    job_id = index  # Assuming job ID is the index
    job_text = row['All']
    
    similarity_scores = TFIDF(resume_df['Processed_Resume'], [job_text])  # Now comparing against resumes
    top = sorted(range(len(similarity_scores)), key=lambda i: similarity_scores[i], reverse=True)[:100]
    
    list_scores = [similarity_scores[i][0][0] for i in top]
    recommendation = get_recommendation_rev(top, resume_df, list_scores)
    
    recommendation["JobID"] = job_id
    all_recommendations_TFIDF_rev.append(recommendation)

print("TFIDF Calculated for Jobs")




TFIDF Calculated for Jobs


In [254]:
# Store and display Recommendations using TFIDF

# final_recommendations = pd.concat(all_recommendations_TFIDF, ignore_index=True)
# final_recommendations.to_csv("user_recommendations_tfidf.csv", index=False)
# print(all_recommendations_TFIDF_rev)
print("Recommendations saved successfully for TFIDF.")


Recommendations saved successfully for TFIDF.


# CV for Recruiter

In [255]:
# This cell calls Count Vectorization for each job to recommend resumes
all_recommendations_CV_rev = []

for index, row in df_combined.iterrows():
    job_id = index
    job_text = row['All']
    
    similarity_scores = count_vectorize(resume_df['Processed_Resume'], [job_text])
    top = sorted(range(len(similarity_scores)), key=lambda i: similarity_scores[i], reverse=True)[:100]
    
    list_scores = [similarity_scores[i][0][0] for i in top]
    recommendation = get_recommendation_rev(top, resume_df, list_scores)
    
    recommendation["JobID"] = job_id
    all_recommendations_CV_rev.append(recommendation)

print("Count Vectorization Calculated for Jobs")


Count Vectorization Calculated for Jobs


In [256]:
# Store and display Recommendations using CV

# final_recommendations = pd.concat(all_recommendations_TFIDF, ignore_index=True)
# final_recommendations.to_csv("user_recommendations_tfidf.csv", index=False)
# print(all_recommendations_CV_rev)
print("Recommendations saved successfully for TFIDF.")


Recommendations saved successfully for TFIDF.


# KNN for Recruiter

In [257]:

# This cell calls KNN for each job to recommend resumes
all_recommendations_KNN_rev = []

for index, row in df_combined.iterrows():
    job_id = index
    job_text = row['All']
    
    recommendation = KNN_rev(resume_df['Processed_Resume'], [job_text])
    
    recommendation["JobID"] = job_id
    all_recommendations_KNN_rev.append(recommendation)

print("KNN Calculated for Jobs")


KNN Calculated for Jobs


In [258]:
# Store and display Recommendations using KNN

# final_recommendations = pd.concat(all_recommendations_TFIDF, ignore_index=True)
# final_recommendations.to_csv("user_recommendations_tfidf.csv", index=False)
# print(all_recommendations_KNN_rev)
print("Recommendations saved successfully for TFIDF.")


Recommendations saved successfully for TFIDF.


# Scale and Assign Weights for Recruiter

In [259]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Convert lists of recommendations into DataFrames
df_TFIDF_rev = pd.concat(all_recommendations_TFIDF_rev, ignore_index=True)
df_CV_rev = pd.concat(all_recommendations_CV_rev, ignore_index=True)
df_KNN_rev = pd.concat(all_recommendations_KNN_rev, ignore_index=True)

# Select only relevant columns before merging
df_TFIDF_rev = df_TFIDF_rev[['JobID', 'user_id', 'score']].rename(columns={'score': 'score_TFIDF'})
df_CV_rev = df_CV_rev[['JobID', 'user_id', 'score']].rename(columns={'score': 'score_CV'})
df_KNN_rev = df_KNN_rev[['JobID', 'user_id', 'score']].rename(columns={'score': 'score_KNN'})

# Merge on 'JobID' and 'user_id' to align resume recommendations for each job
merged_df_rev = df_TFIDF_rev.merge(df_CV_rev, on=["JobID", "user_id"], how="inner")
merged_df_rev = merged_df_rev.merge(df_KNN_rev, on=["JobID", "user_id"], how="inner")

# Scale scores using MinMaxScaler to normalize across methods
scaler = MinMaxScaler()
merged_df_rev[['score_KNN', 'score_TFIDF', 'score_CV']] = scaler.fit_transform(
    merged_df_rev[['score_KNN', 'score_TFIDF', 'score_CV']]
)

# Calculate final weighted score (equal weightage to all)
merged_df_rev['Final_Score'] = (merged_df_rev['score_KNN'] + merged_df_rev['score_TFIDF'] + merged_df_rev['score_CV']) / 3

# Sort results for each job by Final_Score
final_df_rev = merged_df_rev.sort_values(by=["JobID", "Final_Score"], ascending=[True, False])

# Save final recommendations to CSV
final_df_rev.to_csv("final_resume_recommendations.csv", index=False)

# Display top recommendations
print(final_df_rev.head())


  JobID user_id  score_TFIDF  score_CV  score_KNN  Final_Score
0     0     508     0.266280  0.314190   0.266280     0.282250
1     0       3     0.200329  0.272160   0.200329     0.224272
2     0     518     0.197978  0.248424   0.197978     0.214793
3     0       1     0.179213  0.227753   0.179213     0.195393
4     0     506     0.135457  0.197288   0.135457     0.156068


# Top 3 Recommended Resume

In [260]:
# Group by user_id and get top 3 recommendations per user
top_3 = final_df_rev.groupby("JobID").head(3)

# Sort the final result by user_id and Final Score
top_3 = top_3.sort_values(by=["JobID", "Final_Score"], ascending=[True, False])

# Save the final top recommendations to CSV
top_3.to_csv("final_resume_recommendations.csv", index=False)

# Display the result
print(top_3)

     JobID user_id  score_TFIDF  score_CV  score_KNN  Final_Score
0        0     508     0.266280  0.314190   0.266280     0.282250
1        0       3     0.200329  0.272160   0.200329     0.224272
2        0     518     0.197978  0.248424   0.197978     0.214793
8        1     508     0.269614  0.381399   0.269614     0.306876
9        1       2     0.114547  0.204080   0.114547     0.144391
...    ...     ...          ...       ...        ...          ...
7985   998       1     0.369268  0.435215   0.369268     0.391250
7986   998     508     0.320650  0.375711   0.320650     0.339004
7992   999     508     0.294571  0.367929   0.294571     0.319024
7993   999     518     0.220782  0.257017   0.220782     0.232861
7994   999       3     0.200774  0.224693   0.200774     0.208747

[7347 rows x 6 columns]


# Load Interaction Data Set

In [311]:
file_path = current_path+"final.csv"
df_interaction = pd.read_csv(file_path)

# Generate random actual_application values (0 or 1 or 2) based on a probability distribution
# np.random.seed(42)  # For reproducibility

def simulate_user_behavior(df): 
    actual_applications = [] 
    for score in df["Final"]: 
        if score > 0.4: 
            action = np.random.choice([2, 1, 0], p=[0.5, 0.3, 0.2]) # Higher chance of apply 
        elif score > 0.3: 
            action = np.random.choice([1, 2, 0], p=[0.6, 0.2, 0.2]) # More saves, some applies 
        else: 
            action = np.random.choice([0, 1, 2], p=[0.6, 0.3, 0.1]) # Mostly ignored 
        actual_applications.append(action) 
    df["actual_application"] = actual_applications 
    return df # Apply simulation function 
df_interaction = simulate_user_behavior(df_interaction) # Display sample results print(df_interaction.head()) # Save to CSV (Optional) df_interaction.to_csv("simulated_job_interactions.csv", index=False)

# df_interaction["actual_application"] = np.random.choice([0, 1, 2], size=len(df), p=[0.3, 0.3,0.4])  

# Save the updated dataset
output_file = current_path+"interaction_final.csv"
df_interaction.to_csv(output_file, index=False)
df_interaction.head()

,user_id,JobID,score_TFIDF,score_CV,score_KNN,score_BERT,Final,actual_application
0,1,2350,0.185272,0.220928,0.185272,0.544655,0.544655,0
1,1,1688,0.292058,0.324630,0.292058,0.511246,0.511246,2
2,1,2444,0.177338,0.179962,0.177338,0.486068,0.486068,2
3,1,1743,0.124877,0.175399,0.124877,0.484760,0.484760,0
4,1,2321,0.152668,0.190309,0.152668,0.468483,0.468483,0


# Evaluate Content Based Filtering

In [361]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

# Load Data
final_df = pd.read_csv('interaction_final.csv')

# Scale Features
scaler = MinMaxScaler()
final_df[["score_KNN", "score_TFIDF", "score_CV"]] = scaler.fit_transform(final_df[["score_KNN", "score_TFIDF", "score_CV"]])

# Compute Final Score
final_df['Final'] = (
    # final_df['score_KNN'] * 0.2 +
    # final_df['score_TFIDF'] * 0.1 +
    # final_df['score_CV'] * 0.1 +
    final_df['score_BERT'] 
)

# Sort and Select Top 10 Recommendations
final_df = final_df.sort_values(by=["user_id", "Final"], ascending=[True, False])
top_10 = final_df.groupby("user_id").head(10).sort_values(by=["user_id", "Final"], ascending=[True, False])

# Define Evaluation Function
def evaluate_cbf_model(df, k=10, threshold=1):
    def calculate_rmse(y_true, y_pred):
        return np.sqrt(mean_squared_error(y_true, y_pred))

    def precision_at_k(df, k, threshold):
        precision_scores = []
        for user, user_df in df.groupby("user_id"):
            top_k = user_df.nlargest(k, "Final")
            relevant_items = (top_k["actual_application"] >= threshold).sum()
            precision = relevant_items / k
            precision_scores.append(precision)
        return np.mean(precision_scores)

    def recall_at_k(df, k, threshold):
        relevant_items_per_user = df.groupby("user_id")["actual_application"].apply(lambda x: (x >= threshold).sum())
        retrieved_relevant_per_user = df.groupby("user_id").apply(lambda x: (x.nlargest(k, "Final")["actual_application"] >= threshold).sum())
        recall_scores = retrieved_relevant_per_user / relevant_items_per_user.replace(0, np.nan)
        return recall_scores.mean()

    def ndcg_at_k(df, k):
        def dcg(scores):
            return np.sum((2 ** scores - 1) / np.log2(np.arange(2, len(scores) + 2)))

        df_sorted = df.groupby("user_id").apply(lambda x: x.nlargest(k, "Final")).reset_index(drop=True)
        actual_dcg = df_sorted.groupby("user_id")['actual_application'].apply(dcg).mean()

        df_sorted_ideal = df.groupby("user_id").apply(lambda x: x.nlargest(k, "actual_application")).reset_index(drop=True)
        ideal_dcg = df_sorted_ideal.groupby("user_id")['actual_application'].apply(dcg).mean()

        return actual_dcg / ideal_dcg if ideal_dcg > 0 else 0

    def f1_at_k(df, k, threshold):
        precision = precision_at_k(df, k, threshold)
        recall = recall_at_k(df, k, threshold)
        return 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    metrics = {
        "RMSE": calculate_rmse(df['actual_application'], df['Final']),
        "Precision@K": precision_at_k(df, k, threshold),
        "Recall@K": recall_at_k(df, k, threshold),
        "NDCG@K": ndcg_at_k(df, k),
        "F1 Score@K": f1_at_k(df, k, threshold)
    }
    
    return metrics

# Evaluate Model on Top 10 Recommendations
metrics = evaluate_cbf_model(top_10, k=3, threshold=1)

# Print Results
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")


RMSE: 1.0444
Precision@K: 0.7917
Recall@K: 0.3292
NDCG@K: 0.5441
F1 Score@K: 0.4650


C:\Users\vedan\AppData\Local\Temp\ipykernel_29036\3526515921.py:41: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  retrieved_relevant_per_user = df.groupby("user_id").apply(lambda x: (x.nlargest(k, "Final")["actual_application"] >= threshold).sum())
C:\Users\vedan\AppData\Local\Temp\ipykernel_29036\3526515921.py:49: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_sorted = df.groupby("user_id").apply(lambda 

# Random

In [354]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error

def evaluate_model(test_df, pred_df, k=3, threshold=1):
    def calculate_rmse(y_true, y_pred):
        return np.sqrt(mean_squared_error(y_true, y_pred))

    def precision_at_k(df, k, threshold):
        precision_scores = []
        for user, user_df in df.groupby("user_id"):
            top_k = user_df.nlargest(k, "Final_Score")  # Select top K jobs
            relevant_items = (top_k["actual_application"] >= threshold).sum()
            precision = relevant_items / k
            precision_scores.append(precision)
        return np.mean(precision_scores)

    def recall_at_k(df, k, threshold):
        relevant_items_per_user = df.groupby("user_id")["actual_application"].apply(lambda x: (x >= threshold).sum())
        retrieved_relevant_per_user = df.groupby("user_id").apply(lambda x: (x.nlargest(k, "Final_Score")["actual_application"] >= threshold).sum())
        recall_scores = retrieved_relevant_per_user / relevant_items_per_user.replace(0, np.nan)
        return recall_scores.mean()

    def ndcg_at_k(df, k):
        def dcg(scores):
            return np.sum((2 ** scores - 1) / np.log2(np.arange(2, len(scores) + 2)))
        
        df_sorted = df.groupby("user_id").apply(lambda x: x.nlargest(k, "Final_Score")).reset_index(drop=True)
        actual_dcg = df_sorted.groupby("user_id")['actual_application'].apply(dcg).mean()
        
        df_sorted_ideal = df.groupby("user_id").apply(lambda x: x.nlargest(k, "actual_application")).reset_index(drop=True)
        ideal_dcg = df_sorted_ideal.groupby("user_id")['actual_application'].apply(dcg).mean()
        
        return actual_dcg / ideal_dcg if ideal_dcg > 0 else 0

    def f1_at_k(df, k, threshold):
        precision = precision_at_k(df, k, threshold)
        recall = recall_at_k(df, k, threshold)
        return 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    # Compute metrics
    rmse = calculate_rmse(test_df["actual_application"], pred_df["Final_Score"])
    precision = precision_at_k(pred_df, k, threshold)
    recall = recall_at_k(pred_df, k, threshold)
    ndcg = ndcg_at_k(pred_df, k)
    f1_score = f1_at_k(pred_df, k, threshold)

    return {
        "RMSE": round(rmse, 4),
        "Precision@K": round(precision, 4),
        "Recall@K": round(recall, 4),
        "NDCG@K": round(ndcg, 4),
        "F1 Score@K": round(f1_score, 4)
    }

# Generate random model (baseline)
random_df = final_df.copy()
random_df["Final_Score"] = np.random.rand(len(random_df))  # Assign random scores

# Evaluate Random Model
random_results = evaluate_model(final_df, random_df)

# Display results
print(random_results)


C:\Users\vedan\AppData\Local\Temp\ipykernel_29036\1383469710.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  retrieved_relevant_per_user = df.groupby("user_id").apply(lambda x: (x.nlargest(k, "Final_Score")["actual_application"] >= threshold).sum())
C:\Users\vedan\AppData\Local\Temp\ipykernel_29036\1383469710.py:30: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_sorted = df.groupby("user_id").apply(l

{'RMSE': 0.9611, 'Precision@K': 0.5833, 'Recall@K': 0.0507, 'NDCG@K': 0.387, 'F1 Score@K': 0.0932}


C:\Users\vedan\AppData\Local\Temp\ipykernel_29036\1383469710.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  retrieved_relevant_per_user = df.groupby("user_id").apply(lambda x: (x.nlargest(k, "Final_Score")["actual_application"] >= threshold).sum())


# Collaborative Filtering

In [316]:
from surprise import Dataset, Reader
from surprise import KNNBasic
from surprise.model_selection import train_test_split
from surprise.accuracy import rmse

# Load Your Data (Replace with actual file)
df = df_interaction

# Convert actual_application to implicit rating scores
interaction_map = {0: 1, 1: 3, 2: 5}  # Clicked:1, Saved:3, Applied:5
df["rating"] = df["actual_application"].map(interaction_map)

# Define Reader (rating scale 1-5)
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['user_id', 'JobID', 'rating']], reader)

# Split Data into Train & Test
trainset, testset = train_test_split(data, test_size=0.2)

# Use Collaborative Filtering (User-Based KNN)
sim_options = {'name': 'cosine', 'user_based': True}
model = KNNBasic(sim_options=sim_options)
model.fit(trainset)

# Evaluate Model
predictions = model.test(testset)
# rmse_score = rmse(predictions)
# print(f"RMSE: {rmse_score:.3f}")

# Function to Get Job Recommendations
def get_job_recommendations(user_id, df, model, top_n=5):
    job_ids = df['JobID'].unique()
    already_interacted = df[df['user_id'] == user_id]['JobID'].tolist()
    candidate_jobs = [job for job in job_ids if job not in already_interacted]
    # Predict Scores for Candidate Jobs
    predictions = [model.predict(user_id, job).est for job in candidate_jobs]
    recommended_jobs = sorted(zip(candidate_jobs, predictions), key=lambda x: x[1], reverse=True)[:top_n]
    
    return recommended_jobs

# Recommend Jobs for User k
k=508
recommended_jobs = get_job_recommendations(k, df, model)
print(f"Recommended Jobs for User {k}: {recommended_jobs}")



Computing the cosine similarity matrix...
Done computing similarity matrix.
Recommended Jobs for User 508: [(1719, 5), (95, 5), (1264, 5), (1222, 5), (1304, 5)]


In [318]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise.accuracy import rmse

# Load Data
df = df_interaction.copy()

# Convert actual_application to implicit rating scores
interaction_map = {0: 1, 1: 3, 2: 5}  # Clicked:1, Saved:3, Applied:5
df["rating"] = df["actual_application"].map(interaction_map)

# Define Reader (rating scale 1-5)
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['user_id', 'JobID', 'rating']], reader)

# Split Data into Train & Test
trainset, testset = train_test_split(data, test_size=0.2)

# Use SVD for Collaborative Filtering
model = SVD()
model.fit(trainset)

# Evaluate Model
predictions = model.test(testset)
rmse_score = rmse(predictions)
print(f"RMSE: {rmse_score:.3f}")

# Function to Get Job Recommendations
def get_job_recommendations(user_id, df, model, top_n=5):
    job_ids = df['JobID'].unique()
    already_interacted = df[df['user_id'] == user_id]['JobID'].tolist()
    candidate_jobs = [job for job in job_ids if job not in already_interacted]
    
    # Predict Scores for Candidate Jobs
    predictions = [(job, model.predict(user_id, job).est) for job in candidate_jobs]
    recommended_jobs = sorted(predictions, key=lambda x: x[1], reverse=True)[:top_n]
    
    return recommended_jobs

# Recommend Jobs for User k
k = 508
recommended_jobs = get_job_recommendations(k, df, model)
print(f"Recommended Jobs for User {k}: {recommended_jobs}")


RMSE: 1.6627
RMSE: 1.663
Recommended Jobs for User 508: [(1731, 2.846665608523045), (1453, 2.838898404498254), (2308, 2.834273021777654), (516, 2.8241249410779914), (1688, 2.8036553849097747)]


In [348]:
from surprise import accuracy
import numpy as np

def evaluate_cf_model(predictions, k=5):
    # Convert predictions to dictionary format
    # Compute RMSE and MAE
    rmse_score = accuracy.rmse(predictions)
    # mae_score = accuracy.mae(predictions)
    user_est_true = {}
    for uid, iid, true_r, est, _ in predictions:
        if uid not in user_est_true:
            user_est_true[uid] = []
        user_est_true[uid].append((true_r, est))

    precisions, recalls, f1_scores, ndcg_scores = [], [], [], []

    for uid, ratings in user_est_true.items():
        ratings.sort(key=lambda x: x[1], reverse=True)  # Sort by estimated rating
        top_k = ratings[:k]

        relevant_items = sum(1 for true, _ in ratings if true >= 3)  # Relevant if rating ≥ 3
        recommended_relevant = sum(1 for true, _ in top_k if true >= 3)

        precision = recommended_relevant / k
        recall = recommended_relevant / relevant_items if relevant_items > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        dcg = sum((2**true - 1) / np.log2(idx + 2) for idx, (true, _) in enumerate(top_k))
        idcg = sum((2**true - 1) / np.log2(idx + 2) for idx, (true, _) in enumerate(sorted(ratings, key=lambda x: x[0], reverse=True)[:k]))
        ndcg = dcg / idcg if idcg > 0 else 0

        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)
        ndcg_scores.append(ndcg)

    print(f"Precision@{k}: {np.mean(precisions):.4f}")
    print(f"Recall@{k}: {np.mean(recalls):.4f}")
    print(f"F1-score@{k}: {np.mean(f1_scores):.4f}")
    print(f"NDCG@{k}: {np.mean(ndcg_scores):.4f}")



# Compute Precision, Recall, F1-score, and NDCG
evaluate_cf_model(predictions, k=3)


RMSE: 1.6627
Precision@3: 0.5833
Recall@3: 0.3476
F1-score@3: 0.3743
NDCG@3: 0.4345


# Hybrid

In [367]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, precision_score, recall_score
from surprise import SVD, Dataset, Reader
from surprise.model_selection import cross_validate
from scipy.stats import rankdata

# 1️⃣ Load the Data
df = df_interaction  # Ensure it has user_id, JobID, score_TFIDF, score_CV, score_KNN, score_BERT, actual_application

# 2️⃣ Normalize the Scores
scaler = MinMaxScaler()
df[['score_BERT']] = scaler.fit_transform(df[['score_BERT']])

# 3️⃣ Train-Test Split
train, test = train_test_split(df, test_size=0.2, random_state=42)

# 4️⃣ Collaborative Filtering (SVD)
reader = Reader(rating_scale=(0, 2))  # actual_application has values {0 (clicked), 1 (saved), 2 (applied)}
data = Dataset.load_from_df(train[['user_id', 'JobID', 'actual_application']], reader)

svd = SVD()
cross_validate(svd, data, cv=6, verbose=True)

trainset = data.build_full_trainset()
svd.fit(trainset)

# Predict on test set
test['pred_collab'] = test.apply(lambda row: svd.predict(row['user_id'], row['JobID']).est, axis=1)

# 5️⃣ Hybrid Score Calculation (Weighted Average)
# weights = {'TFIDF': 0.1, 'CV': 0.1, 'KNN': 0.1, 'BERT': 0.2, 'Collab': 0.4}
weights = {'Final': 0.5, 'Collab': 0.5}

# test['Final_Score'] = (
#     weights['TFIDF'] * test['score_TFIDF'] + 
#     weights['CV'] * test['score_CV'] + 
#     weights['KNN'] * test['score_KNN'] + 
#     weights['BERT'] * test['score_BERT'] + 
#     weights['Collab'] * test['pred_collab']
# )
test['Final_Score'] = (
    weights['Final'] * test['score_BERT'] + 
    weights['Collab'] * test['pred_collab']
)


Evaluating RMSE, MAE of algorithm SVD on 6 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Mean    Std     
RMSE (testset)    0.7221  0.7698  0.7361  0.7759  0.8268  0.7178  0.7581  0.0378  
MAE (testset)     0.5768  0.6671  0.6333  0.6695  0.7198  0.6117  0.6464  0.0458  
Fit time          0.00    0.02    0.00    0.02    0.01    0.00    0.01    0.01    
Test time         0.02    0.00    0.00    0.00    0.00    0.00    0.00    0.01    


In [334]:
# import pandas as pd
# import numpy as np
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error
# from surprise import SVD, SVDpp, NMF, Dataset, Reader
# from surprise.model_selection import cross_validate
# from scipy.stats import rankdata

# # 1️⃣ Load the Data
# df = df_interaction  # Ensure it has user_id, JobID, score_TFIDF, score_CV, score_KNN, score_BERT, actual_application

# # 2️⃣ Normalize All Scores
# scaler = MinMaxScaler()
# df[['score_TFIDF', 'score_CV', 'score_KNN', 'score_BERT']] = scaler.fit_transform(
#     df[['score_TFIDF', 'score_CV', 'score_KNN', 'score_BERT']]
# )

# # 3️⃣ Train-Test Split
# train, test = train_test_split(df, test_size=0.2, random_state=42)

# # 4️⃣ Collaborative Filtering (SVD++)
# reader = Reader(rating_scale=(0, 2))  # actual_application has values {0 (clicked), 1 (saved), 2 (applied)}
# data = Dataset.load_from_df(train[['user_id', 'JobID', 'actual_application']], reader)

# svd = SVDpp()  # Improved version of SVD
# cross_validate(svd, data, cv=6, verbose=True)

# trainset = data.build_full_trainset()
# svd.fit(trainset)

# # Predict on test set
# test['pred_collab'] = test.apply(lambda row: svd.predict(row['user_id'], row['JobID']).est, axis=1)

# # 5️⃣ Rank Normalization Instead of Direct Weighted Sum
# for col in ['score_TFIDF', 'score_CV', 'score_KNN', 'score_BERT', 'pred_collab']:
#     test[f'rank_{col}'] = rankdata(test[col]) / len(test)

# # 6️⃣ Compute Final Hybrid Score (Tuned Weights)
# weights = {'TFIDF': 0.15, 'CV': 0.1, 'KNN': 0.15, 'BERT': 0.25, 'Collab': 0.35}
# test['Final_Score'] = (
#     weights['TFIDF'] * test['rank_score_TFIDF'] +
#     weights['CV'] * test['rank_score_CV'] +
#     weights['KNN'] * test['rank_score_KNN'] +
#     weights['BERT'] * test['rank_score_BERT'] +
#     weights['Collab'] * test['rank_pred_collab']
# )

# # 7️⃣ Sort Top Recommendations per User
# top_10_recommendations = test.groupby("user_id").apply(lambda x: x.nlargest(10, 'Final_Score')).reset_index(drop=True)

# # Print Example Recommendations
# print(top_10_recommendations.head(10))


In [288]:
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.model_selection import train_test_split
# from surprise import KNNBasic, Dataset, Reader
# from surprise.model_selection import cross_validate

# # 1️⃣ Load Data
# df = df_interaction  # Ensure it has user_id, JobID, score_TFIDF, score_CV, score_KNN, score_BERT, actual_application

# # 2️⃣ Normalize Scores
# scaler = MinMaxScaler()
# df[['score_BERT']] = scaler.fit_transform(df[['score_BERT']])

# # 3️⃣ Train-Test Split
# train, test = train_test_split(df, test_size=0.2, random_state=42)

# # 4️⃣ Collaborative Filtering (KNN)
# reader = Reader(rating_scale=(0, 2))  
# data = Dataset.load_from_df(train[['user_id', 'JobID', 'actual_application']], reader)

# sim_options = {
#     'name': 'cosine',  
#     'user_based': False,  
#     'min_support': 1  # Ensure at least 1 common interaction
# }

# knn = KNNBasic(sim_options=sim_options)
# cross_validate(knn, data, cv=5, verbose=True)

# trainset = data.build_full_trainset()
# knn.fit(trainset)

# # 5️⃣ Predict on Test Set with Exception Handling
# def safe_predict(row):
#     try:
#         return knn.predict(row['user_id'], row['JobID']).est
#     except ZeroDivisionError:
#         return 0  # Assign a neutral score in case of an error

# test['pred_collab'] = test.apply(safe_predict, axis=1)

# # 6️⃣ Hybrid Score Calculation
# weights = {'Final': 0.4, 'Collab': 0.6}

# test['Final_Score'] = (
#     weights['Final'] * test['score_BERT'] + 
#     weights['Collab'] * test['pred_collab']
# )


In [368]:
# def evaluate_hybrid_model(df, k=10, threshold=1):
#     def calculate_rmse(y_true, y_pred):
#         return np.sqrt(mean_squared_error(y_true, y_pred))

#     def precision_at_k(df, k, threshold):
#         precision_scores = []
#         for user, user_df in df.groupby("user_id"):
#             top_k = user_df.nlargest(k, "Final_Score")
#             relevant_items = (top_k["actual_application"] >= threshold).sum()
#             precision = relevant_items / k
#             precision_scores.append(precision)
#         return np.mean(precision_scores)

#     def recall_at_k(df, k, threshold):
#         relevant_items_per_user = df.groupby("user_id")["actual_application"].apply(lambda x: (x >= threshold).sum())
#         retrieved_relevant_per_user = df.groupby("user_id").apply(lambda x: (x.nlargest(k, "Final_Score")["actual_application"] >= threshold).sum())
#         recall_scores = retrieved_relevant_per_user / relevant_items_per_user.replace(0, np.nan)
#         return recall_scores.mean()

#     def ndcg_at_k(df, k):
#         def dcg(scores):
#             return np.sum((2 ** scores - 1) / np.log2(np.arange(2, len(scores) + 2)))

#         df_sorted = df.groupby("user_id").apply(lambda x: x.nlargest(k, "Final_Score")).reset_index(drop=True)
#         actual_dcg = df_sorted.groupby("user_id")['actual_application'].apply(dcg).mean()

#         df_sorted_ideal = df.groupby("user_id").apply(lambda x: x.nlargest(k, "actual_application")).reset_index(drop=True)
#         ideal_dcg = df_sorted_ideal.groupby("user_id")['actual_application'].apply(dcg).mean()

#         return actual_dcg / ideal_dcg if ideal_dcg > 0 else 0

#     def f1_at_k(df, k, threshold):
#         precision = precision_at_k(df, k, threshold)
#         recall = recall_at_k(df, k, threshold)
#         return 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

#     metrics = {
#         "RMSE": calculate_rmse(df['actual_application'], df['Final_Score']),
#         "Precision@K": precision_at_k(df, k, threshold),
#         "Recall@K": recall_at_k(df, k, threshold),
#         "NDCG@K": ndcg_at_k(df, k),
#         "F1 Score@K": f1_at_k(df, k, threshold)
#     }
    
#     return metrics

# # Call the function
# metrics = evaluate_hybrid_model(test, k=10, threshold=1)

# # Print results
# for metric, value in metrics.items():
#     print(f"{metric}: {value:.4f}")


In [372]:
def evaluate_hybrid_model(df, k=10, threshold=1):
    def calculate_rmse(y_true, y_pred):
        return np.sqrt(mean_squared_error(y_true, y_pred))

    def precision_at_k(df, k, threshold):
        precision_scores = []
        for user, user_df in df.groupby("user_id"):
            top_k = user_df.nlargest(k, "Final_Score")  # Get Top-K
            relevant_items = (top_k["actual_application"] >= threshold).sum()
            precision = relevant_items / k
            precision_scores.append(precision)
        return np.mean(precision_scores)

    def recall_at_k(df, k, threshold):
        relevant_items_per_user = df.groupby("user_id")["actual_application"].apply(lambda x: (x >= threshold).sum())
        retrieved_relevant_per_user = df.groupby("user_id").apply(lambda x: (x.nlargest(k, "Final_Score")["actual_application"] >= threshold).sum())
        recall_scores = retrieved_relevant_per_user / relevant_items_per_user.replace(0, np.nan)
        return recall_scores.mean()

    def ndcg_at_k(df, k):
        def dcg(scores):
            return np.sum((2 ** scores - 1) / np.log2(np.arange(2, len(scores) + 2)))

        df_sorted = df.groupby("user_id").apply(lambda x: x.nlargest(k, "Final_Score")).reset_index(drop=True)
        actual_dcg = df_sorted.groupby("user_id")['actual_application'].apply(dcg).mean()

        df_sorted_ideal = df.groupby("user_id").apply(lambda x: x.nlargest(k, "actual_application")).reset_index(drop=True)
        ideal_dcg = df_sorted_ideal.groupby("user_id")['actual_application'].apply(dcg).mean()

        return actual_dcg / ideal_dcg if ideal_dcg > 0 else 0

    def f1_at_k(df, k, threshold):
        precision = precision_at_k(df, k, threshold)
        recall = recall_at_k(df, k, threshold)
        return 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    # ✅ **Filter to Top-10 recommendations per user**
    top_10 = df.groupby("user_id").apply(lambda x: x.nlargest(k, "Final_Score")).reset_index(drop=True)

    metrics = {
        "RMSE": calculate_rmse(df['actual_application'], df['Final_Score']),  # Use full dataset for RMSE
        "Precision@K": precision_at_k(top_10, k, threshold),
        "Recall@K": recall_at_k(top_10, k, threshold),
        "NDCG@K": ndcg_at_k(top_10, k),
        "F1 Score@K": f1_at_k(top_10, k, threshold)
    }
    
    return metrics
# Call the function
metrics = evaluate_hybrid_model(test, k=5, threshold=1)

# Print results
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")


C:\Users\vedan\AppData\Local\Temp\ipykernel_29036\1331784184.py:38: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_10 = df.groupby("user_id").apply(lambda x: x.nlargest(k, "Final_Score")).reset_index(drop=True)
C:\Users\vedan\AppData\Local\Temp\ipykernel_29036\1331784184.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  retrieved_relevant_per_user = df.groupby("user_id").apply(lambda x: (x.nlargest(k,

RMSE: 0.7660
Precision@K: 0.7250
Recall@K: 1.0000
NDCG@K: 0.8145
F1 Score@K: 0.8406


C:\Users\vedan\AppData\Local\Temp\ipykernel_29036\1331784184.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_sorted = df.groupby("user_id").apply(lambda x: x.nlargest(k, "Final_Score")).reset_index(drop=True)
C:\Users\vedan\AppData\Local\Temp\ipykernel_29036\1331784184.py:27: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_sorted_ideal = df.groupby("user_id").apply(lambda x: x.nlargest(k, "actual_a

In [374]:
metrics = evaluate_cbf_model(top_10, k=5, threshold=1)
print("Content Based Filtering Results:")
# Print Results
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")
print("\nCollaborative Filtering Results:")
evaluate_cf_model(predictions, k=5)
print("\nHybrid Model Results:")
metrics = evaluate_hybrid_model(test, k=5, threshold=1)

# Print results
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")


C:\Users\vedan\AppData\Local\Temp\ipykernel_29036\3526515921.py:41: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  retrieved_relevant_per_user = df.groupby("user_id").apply(lambda x: (x.nlargest(k, "Final")["actual_application"] >= threshold).sum())
C:\Users\vedan\AppData\Local\Temp\ipykernel_29036\3526515921.py:49: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_sorted = df.groupby("user_id").apply(lambda 

Content Based Filtering Results:
RMSE: 1.0444
Precision@K: 0.7250
Recall@K: 0.4701
NDCG@K: 0.5525
F1 Score@K: 0.5704

Collaborative Filtering Results:
RMSE: 1.6627
Precision@5: 0.6250
Recall@5: 0.5500
F1-score@5: 0.5317
NDCG@5: 0.4820

Hybrid Model Results:


C:\Users\vedan\AppData\Local\Temp\ipykernel_29036\1331784184.py:38: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_10 = df.groupby("user_id").apply(lambda x: x.nlargest(k, "Final_Score")).reset_index(drop=True)
C:\Users\vedan\AppData\Local\Temp\ipykernel_29036\1331784184.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  retrieved_relevant_per_user = df.groupby("user_id").apply(lambda x: (x.nlargest(k,

RMSE: 0.7660
Precision@K: 0.7250
Recall@K: 1.0000
NDCG@K: 0.8145
F1 Score@K: 0.8406


C:\Users\vedan\AppData\Local\Temp\ipykernel_29036\1331784184.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  retrieved_relevant_per_user = df.groupby("user_id").apply(lambda x: (x.nlargest(k, "Final_Score")["actual_application"] >= threshold).sum())


In [1]:
# import numpy as np
# import pandas as pd
# from sklearn.metrics import mean_squared_error

# def calculate_rmse(y_true, y_pred):
#     return np.sqrt(mean_squared_error(y_true, y_pred))

# def precision_at_k(df, k=5, threshold=1):
#     precision_scores = []
#     for _, user_df in df.groupby("user_id"):
#         top_k = user_df.nlargest(k, "Final_Score")
#         relevant_items = (top_k["actual_application"] >= threshold).sum()
#         precision_scores.append(relevant_items / k)
#     return np.mean(precision_scores)

# def recall_at_k(df, k=5, threshold=1):
#     df_sorted = df.groupby("user_id").apply(lambda x: x.nlargest(k, "Final_Score")).reset_index(drop=True)
#     relevant = (df['actual_application'] >= threshold).sum()
#     retrieved_relevant = (df_sorted['actual_application'] >= threshold).sum()
#     return retrieved_relevant / relevant if relevant > 0 else 0

# def ndcg_at_k(df, k=5):
#     def dcg(scores):
#         return np.sum((2 ** scores - 1) / np.log2(np.arange(2, len(scores) + 2)))
    
#     df_sorted = df.groupby("user_id").apply(lambda x: x.nlargest(k, "Final_Score")).reset_index(drop=True)
#     actual_dcg = df_sorted.groupby("user_id")["actual_application"].apply(dcg).mean()
    
#     df_sorted_ideal = df.groupby("user_id").apply(lambda x: x.nlargest(k, "actual_application")).reset_index(drop=True)
#     ideal_dcg = df_sorted_ideal.groupby("user_id")["actual_application"].apply(dcg).mean()
    
#     return actual_dcg / ideal_dcg if ideal_dcg > 0 else 0

# # Compute RMSE for models
# rmse_cb = calculate_rmse(test['actual_application'], test['Final'])
# rmse_cf = calculate_rmse(test['actual_application'], test['pred_collab'])
# rmse_hybrid = calculate_rmse(test['actual_application'], test['Final_Score'])

# # Compute Precision@K
# precision_cb = precision_at_k(test.assign(Final_Score=test['Final']), k=5)
# precision_cf = precision_at_k(test.assign(Final_Score=test['pred_collab']), k=5)
# precision_hybrid = precision_at_k(test, k=5)

# # Compute Recall@K
# recall_cb = recall_at_k(test.assign(Final_Score=test['Final']), k=5)
# recall_cf = recall_at_k(test.assign(Final_Score=test['pred_collab']), k=5)
# recall_hybrid = recall_at_k(test, k=5)

# # Compute NDCG@K
# ndcg_cb = ndcg_at_k(test.assign(Final_Score=test['Final']), k=5)
# ndcg_cf = ndcg_at_k(test.assign(Final_Score=test['pred_collab']), k=5)
# ndcg_hybrid = ndcg_at_k(test, k=5)

# # Print Results
# print("\n📌 RMSE Comparison:")
# print(f"Content-Based: {rmse_cb:.4f}, Collaborative: {rmse_cf:.4f}, Hybrid: {rmse_hybrid:.4f}")

# print("\n📌 Precision@5 Comparison:")
# print(f"Content-Based: {precision_cb:.4f}, Collaborative: {precision_cf:.4f}, Hybrid: {precision_hybrid:.4f}")

# print("\n📌 Recall@5 Comparison:")
# print(f"Content-Based: {recall_cb:.4f}, Collaborative: {recall_cf:.4f}, Hybrid: {recall_hybrid:.4f}")

# print("\n📌 NDCG@5 Comparison:")
# print(f"Content-Based: {ndcg_cb:.4f}, Collaborative: {ndcg_cf:.4f}, Hybrid: {ndcg_hybrid:.4f}")

